In [1]:
import optuna
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE

c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load training and testing datasets
data_train = pd.read_csv(r"C:\VS code projects\data_files\UNSW_2018_IoT_Botnet_Final_10_best_Training.csv")
data_test = pd.read_csv(r"C:\VS code projects\data_files\UNSW_2018_IoT_Botnet_Final_10_best_Testing.csv")

# Define features (X) and target variable (y) for training and testing sets
X_train = data_train.drop(['category', 'subcategory', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'attack'], axis=1)
y_train = data_train['attack']

X_test = data_test.drop(['category', 'subcategory', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'attack'], axis=1)
y_test = data_test['attack']

print("Training and Testing Datasets Prepared!")

Training and Testing Datasets Prepared!


In [3]:
# Apply SMOTE to balance the class distribution in the training dataset
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Display the class distribution after resampling
print(f"Original y_train distribution:\n{y_train.value_counts()}")
print(f"Resampled y_train distribution:\n{y_train_resampled.value_counts()}")

Original y_train distribution:
attack
1    2934447
0        370
Name: count, dtype: int64
Resampled y_train distribution:
attack
1    2934447
0    2934447
Name: count, dtype: int64


In [4]:
def objective(trial):
    """
    Objective function for Optuna's Bayesian optimization.
    Args:
        trial: Optuna trial object for sampling hyperparameters.
    Returns:
        Accuracy score on the test dataset.
    """
    # Define hyperparameter search space
    param_grid = {
        'objective': 'binary',  # Binary classification
        'metric': 'binary_logloss',  # Loss metric
        'verbosity': -1,  # Suppress training logs
        'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
        'random_state': 42,  # Ensure reproducibility
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 50)
    }

    # Train LightGBM model using hyperparameters sampled by Optuna
    model = lgb.LGBMClassifier(**param_grid)
    model.fit(
        X_train_resampled,
        y_train_resampled,
        eval_set=[(X_test, y_test)],
        eval_metric='logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]  # Early stopping
    )

    # Predict on the testing dataset
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [5]:
import optuna
import optuna.visualization as vis

# Create Optuna study to maximize accuracy
study = optuna.create_study(direction='maximize', study_name='lightgbm_training_testing')
study.optimize(objective, n_trials=50, timeout=3600)  # Run for 50 trials or 1 hour

# Print the best trial results
print("\nBest trial:")
print(f"  Value: {study.best_trial.value:.4f}")
print("  Params:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

# ---- Visualizations ----

# 1. Optimization History
fig1 = vis.plot_optimization_history(study)
fig1.show()

# 2. Hyperparameter Importance
fig2 = vis.plot_param_importances(study)
fig2.show()

# 3. Slice Plot (shows impact of individual parameters)
fig3 = vis.plot_slice(study)
fig3.show()

# 4. Parallel Coordinates Plot (shows interactions between hyperparameters)
fig4 = vis.plot_parallel_coordinate(study)
fig4.show()

# 5. Contour Plot (visualizes relationships between two hyperparameters)
fig5 = vis.plot_contour(study)
fig5.show()

[I 2025-05-09 21:56:33,466] A new study created in memory with name: lightgbm_training_testing
[I 2025-05-09 21:56:50,254] Trial 0 finished with value: 1.0 and parameters: {'learning_rate': 0.016065205094981324, 'num_leaves': 56, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.7572876334719514, 'colsample_bytree': 0.7753545078516937, 'reg_alpha': 3.1067877226474795e-07, 'reg_lambda': 6.2560353478602936e-06, 'n_estimators': 56, 'feature_fraction': 0.43604344159336933, 'bagging_fraction': 0.8617436259493026, 'min_child_weight': 47}. Best is trial 0 with value: 1.0.
[I 2025-05-09 21:57:22,282] Trial 1 finished with value: 1.0 and parameters: {'learning_rate': 0.023638700304826385, 'num_leaves': 55, 'max_depth': 3, 'min_child_samples': 91, 'subsample': 0.8947039858984942, 'colsample_bytree': 0.6388750859772981, 'reg_alpha': 0.00032093877013971697, 'reg_lambda': 1.3039255670389288e-06, 'n_estimators': 216, 'feature_fraction': 0.87984455494484, 'bagging_fraction': 0.420345354925196, 

KeyboardInterrupt: 

In [ ]:
# Save the best hyperparameters and accuracy score to a file
output_file = "models_and_data/best_hyperparameters_lightgbm_training_testing.txt"
with open(output_file, "w") as f:
    f.write("Best Hyperparameters:\n")
    for key, value in study.best_trial.params.items():
        f.write(f"{key}: {value}\n")
    f.write(f"\nBest Accuracy: {study.best_trial.value:.4f}\n")

print(f"\nResults saved to {output_file}")


Results saved to models_and_data/best_hyperparameters_lightgbm_training_testing.txt
